In [1]:
import tensorflow as tf
import os
from keras.layers import (Input, Dense, Flatten, Conv2D, MaxPooling2D, Dropout)
from keras.models import Model, load_model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
import time

In [29]:
datapath = '../GTZAN_Dataset'
genres = list(os.listdir(f'{datapath}/images_original/'))
genres.remove('.DS_Store')
print(genres)

['pop', 'metal', 'disco', 'blues', 'reggae', 'classical', 'rock', 'hiphop', 'country', 'jazz']


In [151]:
#Data has been pre split into test and training folders

data_gen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)
path=f'{datapath}/images_original' 
image_datagentrain=data_gen.flow_from_directory(path,target_size=(300,300),class_mode='categorical')

Found 699 images belonging to 10 classes.


In [43]:
path=f'{datapath}/images_test'
image_datagentest=data_gen.flow_from_directory(path,target_size=(300,300),class_mode='categorical')

Found 300 images belonging to 10 classes.


In [5]:
def GenreModel(input_shape=(300,300,4), classes=10, model_name='GenreModel_'):
    
    model_name=model_name + str(int(time.time()))[4:]
    tensorboard = TensorBoard(log_dir=f'logs/{model_name}')

    X_input = Input(input_shape)
    
    X = (Conv2D(filters=8, kernel_size=(3,3), padding='same', activation='relu', input_shape=(300, 300, 3)))(X_input)
    X = (MaxPooling2D(pool_size=(2,2), strides=(2,2)))(X)

    X = (Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='relu',))(X)
    X = (MaxPooling2D(pool_size=(2,2), strides=(2,2)))(X)

    X = (Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu',))(X)
    X = (MaxPooling2D(pool_size=(2,2), strides=(2,2)))(X)

    X = (Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu',))(X)
    X = (MaxPooling2D(pool_size=(2,2), strides=(2,2)))(X)

    X = Flatten()(X)

    X = Dropout(rate=0.3)(X)

    X = (Dense(128, activation='relu'))(X)
    X = (Dense(64, activation='relu'))(X)
    X = (Dense(10, activation='softmax'))(X)

    model = Model(X_input,X)
    
    return model, tensorboard

model, tensorboard = GenreModel(input_shape=(300,300,3),classes=10)

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0005), metrics=['accuracy'])

history = model.fit(image_datagentrain,
                    epochs=25,
                    callbacks = [tensorboard],         # Remember to add batch size back into argument
                    validation_data=image_datagentest).history

Epoch 1/25
 1/22 [>.............................] - ETA: 0s - loss: 2.3114 - accuracy: 0.0312WARNING:tensorflow:From /Users/jayvier/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
22/22 [==============================] - 12s 560ms/step - loss: 2.2598 - accuracy: 0.1645 - val_loss: 2.0995 - val_accuracy: 0.2600
Epoch 2/25
22/22 [==============================] - 12s 566ms/step - loss: 1.9692 - accuracy: 0.2732 - val_loss: 1.7860 - val_accuracy: 0.3333
Epoch 3/25
22/22 [==============================] - 13s 570ms/step - loss: 1.6992 - accuracy: 0.3820 - val_loss: 1.6070 - val_accuracy: 0.3833
Epoch 4/25
22/22 [==============================] - 13s 569ms/step - loss: 1.5647 - accuracy: 0.4649 - val_loss: 1.4616 - val_accuracy: 0.4633
Epoch 5/25
22/22 [==============================] 

In [6]:
loss, accuracy = model.evaluate(image_datagentest)
print('loss: ', loss, 'accuracy: ', accuracy)

10/10 [==============================] - 1s 138ms/step - loss: 1.2782 - accuracy: 0.6300
loss:  1.2781866788864136 accuracy:  0.6299999952316284


### TensorBoard
We can view an overview of our results in tensorboard.

In [157]:
from tensorboard import notebook

%reload_ext tensorboard
# Command line in current working dir "tensorboard --logdir='logs/'" to start tensorboard
%tensorboard --logdir {'logs/'}

## Optimizing Model structure
Now that we have a basic model that works using hyperas we will find the optimal number of layers and layer sizes for the model, using tensorboard to visualise the results. It is possible that the current model is already optimal. To ealuate the models performance we will use the validation loss parameter. The hyperas library uses hyperopt, and allows us to feed the model a range fo possible parameters such as number of layers and number of nodes per layer.

In [32]:
import hyperas
from hyperas import optim
from hyperas.distributions import choice
from hyperopt import tpe, STATUS_OK, Trials

In [141]:
def GenreModel_optimize(X_train, Y_train, X_test, Y_test):
    

    X_input = Input((300,300,3))
    
    X = (Conv2D({{choice([8,16])}}, kernel_size=(3,3), padding='same', activation='relu', input_shape=(300, 300, 3)))(X_input)
    X = (MaxPooling2D(pool_size=(2,2), strides=(2,2)))(X)

    X = (Conv2D({{choice([8,16,32])}}, kernel_size=(3,3), padding='same', activation='relu',))(X)
    X = (MaxPooling2D(pool_size=(2,2), strides=(2,2)))(X)

    X = (Conv2D({{choice([16,32,64])}}, kernel_size=(3,3), padding='same', activation='relu',))(X)
    X = (MaxPooling2D(pool_size=(2,2), strides=(2,2)))(X)
    
    X = (Conv2D({{choice([32,64,128])}}, kernel_size=(3,3), padding='same', activation='relu',))(X)
    X = (MaxPooling2D(pool_size=(2,2), strides=(2,2)))(X)
    
    nb_conv = {{choice(['four','five'])}}
    if nb_conv == 'five':
        X = (Conv2D({{choice([64,128,256])}}, kernel_size=(3,3), padding='same', activation='relu',))(X)
        X = (MaxPooling2D(pool_size=(2,2), strides=(2,2)))(X)

    X = Flatten()(X)

    X = Dropout(rate=0.3)(X)
    
    nb_dense = {{choice(['one','two','three'])}}
    X = (Dense({{choice([64,128,256,512])}}, activation='relu'))(X)
    if nb_dense == 'two' or 'three':
        X = (Dense({{choice([32,64,128,256])}}, activation='relu'))(X)
        if nb_dense == 'three':
            X = (Dense({{choice([32,64,128,256])}}, activation='relu'))(X)

    
    X = (Dense(10, activation='softmax'))(X)

    model = Model(X_input,X)
    
    model.compile(loss='categorical_crossentropy',
                 optimizer=Adam(learning_rate=0.0005),
                 metrics=['accuracy'])
    
    history = model.fit(X_train,
                        epochs=25,
                        validation_data=X_test).history
    
    
    
    return {'loss' : history['val_loss'][-1], 'status' : STATUS_OK, 'model' : model}

Hyperas requires a data function to feed to the `optim.minimize` funciton.

In [144]:
def data():
    nb_classes = 10
    
    data_gen=ImageDataGenerator(rescale=1./255.)

    X_train = data_gen.flow_from_directory('../GTZAN_Dataset/images_original',target_size=(300,300),batch_size=32,class_mode='categorical')
    X_test = data_gen.flow_from_directory('../GTZAN_Dataset/images_test',target_size=(300,300),batch_size=32,class_mode='categorical')
    
    y_train = X_train.classes
    y_test = X_test.classes
    
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)
    return X_train, Y_train, X_test, Y_test


We optimize the model using the optim function from hyperas.

In [145]:
X_train, Y_train, X_test, Y_test = data()

trials=Trials()

best_run, best_model = optim.minimize(model=GenreModel_optimize,
                                          data=data,
                                          max_evals=1,
                                          algo=tpe.suggest,
                                          notebook_name='CNNFinal',
                                          trials=trials)

print(best_model)

Found 699 images belonging to 10 classes.
Found 300 images belonging to 10 classes.
>>> Imports:
#coding=utf-8

try:
    import tensorflow as tf
except:
    pass

try:
    import os
except:
    pass

try:
    from keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, Dropout
except:
    pass

try:
    from keras.models import Model, load_model
except:
    pass

try:
    from keras.optimizers import Adam
except:
    pass

try:
    from keras.preprocessing.image import ImageDataGenerator
except:
    pass

try:
    from keras.callbacks import TensorBoard
except:
    pass

try:
    import time
except:
    pass

try:
    from tensorboard import notebook
except:
    pass

try:
    import hyperas
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice
except:
    pass

try:
    from hyperopt import tpe, STATUS_OK, Trials
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import sklearn
except:
 

 - 15s 678ms/step - loss: 2.3102 - accuracy: 0.0930 - val_loss: 2.2974 - val_accuracy: 0.1200

Epoch 2/25                                           
 1/22 [>.............................]               
 - ETA: 0s - loss: 2.2945 - accuracy: 0.1250         
                                                    
 2/22 [=>............................]               
 - ETA: 5s - loss: 2.2935 - accuracy: 0.1525         
                                                    
 3/22 [===>..........................]               
 - ETA: 7s - loss: 2.2936 - accuracy: 0.1758         
                                                    
 4/22 [====>.........................]               
 - ETA: 8s - loss: 2.2924 - accuracy: 0.1789         
                                                    
 5/22 [=====>........................]               
 - ETA: 8s - loss: 2.2940 - accuracy: 0.1613         
                                                    
 6/22 [=======>......................]        

 - ETA: 8s - loss: 2.0207 - accuracy: 0.2195         
                                                    
 5/22 [=====>........................]               
 - ETA: 8s - loss: 2.0001 - accuracy: 0.2258         
                                                    
 6/22 [=======>......................]               
 - ETA: 8s - loss: 1.9802 - accuracy: 0.2460         
                                                    
 7/22 [========>.....................]               
 - ETA: 8s - loss: 1.9782 - accuracy: 0.2466         
                                                    
 8/22 [=========>....................]               
 - ETA: 7s - loss: 2.0108 - accuracy: 0.2351         
                                                    
 9/22 [===========>..................]               
 - ETA: 7s - loss: 2.0290 - accuracy: 0.2297         
                                                    
10/22 [============>.................]               
 - ETA: 6s - loss: 2.0042 - accura

                                                    
 9/22 [===========>..................]               
 - ETA: 7s - loss: 1.7769 - accuracy: 0.3229         
                                                    
10/22 [============>.................]               
 - ETA: 7s - loss: 1.7846 - accuracy: 0.3156         
                                                    
11/22 [==============>...............]               
 - ETA: 6s - loss: 1.7704 - accuracy: 0.3125         
                                                    
12/22 [===============>..............]               
 - ETA: 6s - loss: 1.7723 - accuracy: 0.3177         
                                                    
13/22 [================>.............]               
 - ETA: 5s - loss: 1.7750 - accuracy: 0.3197         
                                                    
14/22 [==================>...........]               
 - ETA: 5s - loss: 1.7560 - accuracy: 0.3304         
                                  

13/22 [================>.............]               
 - ETA: 5s - loss: 1.5624 - accuracy: 0.4423         
                                                    
14/22 [==================>...........]               
 - ETA: 4s - loss: 1.5708 - accuracy: 0.4397         
                                                    
15/22 [===================>..........]               
 - ETA: 4s - loss: 1.5648 - accuracy: 0.4354         
                                                    
16/22 [====================>.........]               
 - ETA: 3s - loss: 1.5512 - accuracy: 0.4414         
                                                    
17/22 [======================>.......]               
 - ETA: 3s - loss: 1.5391 - accuracy: 0.4397         
                                                    
18/22 [=======================>......]               
 - ETA: 2s - loss: 1.5259 - accuracy: 0.4518         
                                                    
19/22 [========================>..

...

 7/22 [========>.....................]               
 - ETA: 8s - loss: 1.0104 - accuracy: 0.5938         
                                                    
 8/22 [=========>....................]               
 - ETA: 7s - loss: 1.0216 - accuracy: 0.6055         
                                                    
 9/22 [===========>..................]               
 - ETA: 7s - loss: 0.9961 - accuracy: 0.6146         
                                                    
10/22 [============>.................]               
 - ETA: 6s - loss: 0.9857 - accuracy: 0.6095         
                                                    
11/22 [==============>...............]               
 - ETA: 6s - loss: 0.9645 - accuracy: 0.6196         
                                                    
12/22 [===============>..............]               
 - ETA: 5s - loss: 0.9837 - accuracy: 0.6174         
                                                    
13/22 [================>..........

 - ETA: 6s - loss: 0.8734 - accuracy: 0.6945         
                                                    
12/22 [===============>..............]               
 - ETA: 5s - loss: 0.8672 - accuracy: 0.7018         
                                                    
13/22 [================>.............]               
 - ETA: 5s - loss: 0.8897 - accuracy: 0.6910         
                                                    
14/22 [==================>...........]               
 - ETA: 4s - loss: 0.8895 - accuracy: 0.6840         
                                                    
15/22 [===================>..........]               
 - ETA: 4s - loss: 0.8873 - accuracy: 0.6863         
                                                    
16/22 [====================>.........]               
 - ETA: 3s - loss: 0.8764 - accuracy: 0.6903         
                                                    
17/22 [======================>.......]               
 - ETA: 3s - loss: 0.8846 - accura

                                                    
16/22 [====================>.........]               
 - ETA: 3s - loss: 0.6407 - accuracy: 0.7850         
                                                    
17/22 [======================>.......]               
 - ETA: 3s - loss: 0.6381 - accuracy: 0.7848         
                                                    
18/22 [=======================>......]               
 - ETA: 2s - loss: 0.6393 - accuracy: 0.7846         
                                                    
19/22 [========================>.....]               
 - ETA: 1s - loss: 0.6370 - accuracy: 0.7877         
                                                    
20/22 [==========================>...]               
 - ETA: 1s - loss: 0.6376 - accuracy: 0.7874         
                                                    
21/22 [===========================>..]               
 - ETA: 0s - loss: 0.6364 - accuracy: 0.7871         
                                  

20/22 [==========================>...]               
 - ETA: 1s - loss: 0.5506 - accuracy: 0.8110         
                                                    
21/22 [===========================>..]               
 - ETA: 0s - loss: 0.5375 - accuracy: 0.8141         
                                                    
22/22 [==============================]               
 - ETA: 0s - loss: 0.5304 - accuracy: 0.8169         
                                                    
22/22 [==============================]               
 - 16s 720ms/step - loss: 0.5304 - accuracy: 0.8169 - val_loss: 1.6192 - val_accuracy: 0.5733

Epoch 24/25                                          
 1/22 [>.............................]               
 - ETA: 0s - loss: 0.3836 - accuracy: 0.8750         
                                                    
 2/22 [=>............................]               
 - ETA: 6s - loss: 0.4019 - accuracy: 0.8594         
                                             

100%|██████████| 1/1 [06:50<00:00, 410.41s/trial, best loss: 1.9205126762390137]


We see the results of the optimization by looking at the best run.

In [150]:
# 'Conv2D': hp.choice('Conv2D', [8,16]),
# 'Conv2D_1': hp.choice('Conv2D_1', [8,16,32]),
# 'Conv2D_2': hp.choice('Conv2D_2', [16,32,64]),
# 'Conv2D_3': hp.choice('Conv2D_3', [32,64,128]),
# 'nb_conv': hp.choice('nb_conv', ['four','five']),
# 'Conv2D_4': hp.choice('Conv2D_4', [64,128,256]),
# 'nb_dense': hp.choice('nb_dense', ['one','two','three']),
# 'Dense': hp.choice('Dense', [64,128,256,512]),
# 'Dense_1': hp.choice('Dense_1', [32,64,128,256]),
# 'Dense_2': hp.choice('Dense_2', [32,64,128,256]),
best_run

{'Conv2D': 0,
 'Conv2D_1': 2,
 'Conv2D_2': 1,
 'Conv2D_3': 1,
 'Conv2D_4': 1,
 'Dense': 3,
 'Dense_1': 2,
 'Dense_2': 0,
 'nb_conv': 1,
 'nb_dense': 2}